In [1]:
import numpy as np
import pulp as op
import pandas as pd


def allocatePlanting(p,w,v,L,Q)->(pd.DataFrame(),np.ndarray,np.ndarray,np.ndarray):
    """    """
    n = v.shape[0] #Cantidad de variedades.
    m = w.shape[0] #Cantidad de camas.
    concat = lambda t,e : str(t)+str(e)
    beds = [concat(xt,xe) for xt,xe in zip(np.repeat(['Bed'],m*n),np.repeat(np.arange(1,m+1),n))]
    varieties = [concat(xt,xe) for xt,xe in zip(np.repeat(['_Var'],m*n),np.tile(np.arange(1,n+1),m))]
    bedsXvars = [concat(xt,xe) for xt,xe in zip(beds,varieties)]
    c=dict(zip(bedsXvars, np.tile(p,m)*np.repeat(w,n)))
    A=dict(zip(bedsXvars,np.repeat(w,n)))
    ##Plantear el problema de optimización
    prob = op.LpProblem("Plannting_plan",op.LpMaximize)
    beds = op.LpVariable.dicts("BedsVar",bedsXvars,lowBound=0,upBound=1,cat=op.LpContinuous) #Variables de decisión (Cantidad de esquejes por cama)
    prob += op.lpSum([c[i]*beds[i] for i in bedsXvars]) # Función de Costo, Precio de la variedad * X

    #################
    # RESTRICCIONES #
    #################

    #Capacidad de los bancos por variedad
    for j in np.arange(0,n):
        rTemp = np.zeros(m*n)
        for h in np.arange(0,m):
            p1 = np.zeros(j) #Cantidad de ceros adelante segun el indice de la cama
            p2 = [w[h]] #Capacidad del banco
            p3 = np.zeros(n-(j+1)) #
            rTemp[h*n:(h*n)+n]=np.concatenate((p1,p2,p3))
        r = dict(zip(bedsXvars,rTemp))
        prob += op.lpSum([r[i]*beds[i] for i in bedsXvars])<=v[j] #Restricción del área de la siembra.
    
    #Solo puede sembrarse una variedad por cama.
    for k in np.arange(0,m): #Se crea un restricción por cama.
        p1 = np.zeros(k*n) #Cantidad de ceros adelante segun el indice de la cama
        p2 = np.ones(n) #Cantidad de unos que representan la restriccion
        p3 = np.zeros(((m*n)-((k*n)+n))) #cantidad de ceros al final
        r = dict(zip(bedsXvars,np.concatenate((p1,p2,p3))))
        prob += op.lpSum([r[i]*beds[i] for i in bedsXvars]) <= 1 #Restricción de máximo una variedad por cama.
    
    q = np.unique(Q)
    qCount = [len((Q==xq).nonzero()[0]) for xq in q]
    f_zq = 0
    for c_qi in np.arange(0,len(q)):
        qi=q[c_qi]
        for li in L:
            p1 = np.zeros(n*f_zq)
            p2 = np.tile(li,qCount[c_qi])
            p3 = np.zeros(n*(len(Q)-qCount[c_qi]-f_zq))
            r = dict(zip(bedsXvars,np.concatenate((p1,p2,p3))))
            prob += op.lpSum([r[i]*beds[i] for i in bedsXvars]) == 0
        f_zq += qCount[c_qi]

    #Solución al problema de optimización.
    prob.solve()
    
    #print("Status:",op.LpStatus[prob.status])
    #print("Objective:",op.value(prob.objective))
    #for va in prob.variables():
        #if va.varValue>0:
            #print(va.name, "=", va.varValue)
        
    plantingPlan = np.around((np.array([ x.varValue for x in prob.variables()]) * np.repeat(w,n)),decimals=0)
    bedsT = [concat(xt,xe) for xt,xe in zip(np.repeat(['Bed'],m),np.arange(1,m+1))]
    varsT = [concat(xt,xe) for xt,xe in zip(np.repeat(['Var'],m),np.arange(1,n+1))]
    
    planShaped = plantingPlan.reshape(m,n)
    
    v_Planted = np.sum(planShaped,axis=0)
    
    w_Planted = np.sum(planShaped,axis=1)
    
    
    return (pd.DataFrame(planShaped,index=bedsT, columns=varsT), v-v_Planted, w-w_Planted, plantingPlan)


In [2]:
import requests
import os
import json
import numpy as np

def getWFEndpointsSEPEnabled()->np.ndarray:
    """Funcion que consulta LE API para validar las empresas que tienen el SEP habilitado."""
    
    url = os.environ['LEServiceEndpoint']
    params = {
        "Session":{
		"SessionId":"",
		"UserId":"",
		"SystemClientId":"",
		"InstanceCode":"",
		"Culture":""},
        "ServiceName":"BaseDataAccess",
        "MethodName":"ExecuteProcedure",
        "Params":[{"Name":"args",
                    "Value":{
                        "Name":"[System].[GetEndpointValueForAppSettingEnabled]",
                        "Parameters":[{"Name":"SettingValue","Value":"SEPAppEnabled","SerializedValue":"true","EncryptedValue":"true","Direction":0}]
                        },"Direction":1,"SerializedValue":"true"
                }],
        "FullName":"",
        "Serialized":"true",
        "SerializerType":"GloboStudio.Core.Serialization.JsonSerializer, GloboStudio.Core.Serialization",
        "RequestID":""
    }
    r = requests.post(url=url,data=json.dumps(params))
    return np.array(json.loads(r.json()['d'])['Value']['Data'][0]['Rows'])

def getWFProjectedPlanting(url:str)->np.ndarray:
    """
    Consulta las variedades con la necesidad de siembra ordenadas por el precio.
    
    Parameters:
        url : (string) URL del servicio web en donde se ejecutará la consulta.
    """
    params = {
        "Session":{
		"SessionId":"",
		"UserId":"",
		"SystemClientId":"",
		"InstanceCode":"",
		"Culture":""},
        "ServiceName":"BaseDataAccess",
        "MethodName":"ExecuteProcedure",
        "Params":[{"Name":"args",
                    "Value":{
                        "Name":"[dbo].[GetSEPProjectedPlanting]",
                        "Parameters":[]
                        },"Direction":1,"SerializedValue":"true"
                }],
        "FullName":"",
        "Serialized":"true",
        "SerializerType":"GloboStudio.Core.Serialization.JsonSerializer, GloboStudio.Core.Serialization",
        "RequestID":""
    }
    r = requests.post(url=url,data=json.dumps(params))
    return np.array(json.loads(r.json()['d'])['Value']['Data'][0]['Rows'])

def getWFBedsForPlanting(url:str)->np.ndarray:
    """
    Consulta los bancos que están disponibles para la siembra.
    
    Parameters:
        url : (string) URL del servicio web en donde se ejecutará la consulta.
    """
    params = {
        "Session":{
		"SessionId":"",
		"UserId":"",
		"SystemClientId":"",
		"InstanceCode":"",
		"Culture":""},
        "ServiceName":"BaseDataAccess",
        "MethodName":"ExecuteProcedure",
        "Params":[{"Name":"args",
                    "Value":{
                        "Name":"[dbo].[GetSEPBedsForPlanting]",
                        "Parameters":[]
                        },"Direction":1,"SerializedValue":"true"
                }],
        "FullName":"",
        "Serialized":"true",
        "SerializerType":"GloboStudio.Core.Serialization.JsonSerializer, GloboStudio.Core.Serialization",
        "RequestID":""
    }
    r = requests.post(url=url,data=json.dumps(params))
    return np.array(json.loads(r.json()['d'])['Value']['Data'][0]['Rows'])

def updateWFPlantingSuggestion(url:str, data:str)->np.ndarray:
    """
    Actualiza la estructura de sugerencia de siembra usada en SEP
    
    Parameters:
        url : (string) URL del servicio web en donde se ejecutará la consulta.
    """
    params = {
        "Session":{
		"SessionId":"",
		"UserId":"",
		"SystemClientId":"",
		"InstanceCode":"",
		"Culture":""},
        "ServiceName":"BaseDataAccess",
        "MethodName":"ExecuteProcedure",
        "Params":[{"Name":"args",
                    "Value":{
                        "Name":"[dbo].[UpdateSEPPlantingSuggestion]",
                        "Parameters":[{"Name":"SuggestionData","Value":data,"SerializedValue":"true","EncryptedValue":"true","Direction":0}]
                        },"Direction":1,"SerializedValue":"true"
                }],
        "FullName":"",
        "Serialized":"true",
        "SerializerType":"GloboStudio.Core.Serialization.JsonSerializer, GloboStudio.Core.Serialization",
        "RequestID":""
    }
    r = requests.post(url=url,data=json.dumps(params))
    return np.array(json.loads(r.json()['d'])['Value']['Data'][0]['Rows'])

In [3]:
companyURL = 'http://webflowerspruebasqf.azurewebsites.net/winwebservices/WinWebServiceCommand.svc/ExecuteCommandStream'

In [4]:
vTEMP = getWFProjectedPlanting(companyURL)
wTEMP = getWFBedsForPlanting(companyURL)

In [5]:
wTEMP

array([['6915', '001:01:01:1', '001', '1125.0'],
       ['6917', '001:01:01:2', '001', '1125.0'],
       ['6919', '001:01:01:3', '001', '1125.0'],
       ['6921', '001:01:01:4', '001', '1125.0'],
       ['6923', '001:01:01:5', '001', '1125.0'],
       ['6925', '001:01:01:6', '001', '1125.0']], dtype='<U11')

In [6]:
vTEMP

array([[  406, 22500,     1]])

In [7]:
p = vTEMP[:,2].astype(int)
v = vTEMP[:,1].astype(float)
w = wTEMP[:,3].astype(float)
L = np.array([])
Q = wTEMP[:,2]

In [8]:
allocation, v_pending, w_free, plan = allocatePlanting(p,w,v,L,Q)

In [9]:
f = np.zeros(len(plan)*3).reshape((len(plan),3))
f[:,0]=np.repeat(wTEMP[:,0],len(vTEMP)).astype(str)
f[:,1]=np.tile(vTEMP[:,0],len(wTEMP)).astype(str)
f[:,2]=plan

In [10]:
updateWFPlantingSuggestion(companyURL,json.dumps(f.tolist()))

array([], dtype=float64)

In [11]:
f

array([[6915.,  406., 1125.],
       [6917.,  406., 1125.],
       [6919.,  406., 1125.],
       [6921.,  406., 1125.],
       [6923.,  406., 1125.],
       [6925.,  406., 1125.]])